In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cand1 = pd.read_csv('delf-scored-candidates.csv', index_col=0)

print(len(cand1))
cand1.head()

In [ ]:
cand1.tail()

In [ ]:
cands = []
cands.append(pd.read_csv('delf-scored-candidates-p2.csv', index_col=0))
cands[-1].head()

In [ ]:
cands_combined = cand1.copy()
cands_combined = cands_combined.fillna(-1).sort_index()

for i, cand2 in enumerate(cands):
    cand2 = pd.concat([cand2, cands_combined[~cands_combined.index.isin(cand2.index)]])
    cand2_fn = cand2.fillna(-1).sort_index()
    cands_combined = cands_combined.where(cand2_fn < cands_combined, cand2_fn)
cands_combined.head()

In [ ]:
cands_combined.tail()

In [ ]:
print(len(cands_combined))
print(cand1.isnull().sum())
print(cands_combined.isnull().sum())
print('Negative entries:', (cand1['delf_max_score']<0).sum())

In [ ]:
cands_combined['delf_max_score'] = cands_combined[
    'delf_max_score']/cands_combined['delf_max_score'].max()
cands_combined['delf_mean_score'] = cands_combined[
    'delf_mean_score']/cands_combined['delf_mean_score'].max()
cands_combined['delf_m2_score'] = cands_combined[
    'delf_m2_score']/cands_combined['delf_m2_score'].max()

cands_combined.head()

To understand the value of `nn_w` set in the next cell, it is important to realize that most normalized delf scores have values between 0 and about 0.2. Thus, like this, the NN confidences and the delf scores are weighted roughly 50:50 unless the delf score is exceptionally high.

In [ ]:
nn_w = 0.15

score = 'delf_max_score'
predictions = pd.DataFrame(columns=['landmarks'], index=cands_combined.index)
predictions['landmarks'] = [str(int(tp))+' '+ '%.16g' % pp 
                          for tp,pp in zip(cands_combined['pred_id'].values, 
                                           ((1.-nn_w) * cands_combined[score].values 
                                            + nn_w * cands_combined['nn_conf'].values))]
predictions.head()

In [ ]:
test_info_full = pd.read_csv('test.csv', index_col=0)
print(len(test_info_full))
test_info_full.head()

In [ ]:
missing = test_info_full[test_info_full.index.isin(cands_combined.index)!=True]
missing_predictions = pd.DataFrame(index=missing.index)
missing_predictions['landmarks'] = '9633 0.0'
print(len(missing_predictions))
missing_predictions.head()

In [ ]:
completed_predictions = pd.concat([predictions, missing_predictions])
print(len(completed_predictions))

In [ ]:
sorted_predictions = pd.DataFrame(index=test_info_full.index)
sorted_predictions['landmarks' ] = completed_predictions['landmarks']
sorted_predictions.tail()

In [ ]:
sorted_predictions.to_csv('prediction_reweighted.csv')